In [19]:
train_ratio = 0.85
n_epochs = 30
loss_fn = 'mse'

horizon_pred = 1
seq_length = 30
batch_size = 30


units_layer2 = None

params = {
    'units_layer1' : [100, 200],
    'activation_layer1' : ['relu'],
    'dropout_layer1' : [0.1, 0.2],
    'kernel_reg_factor' : [0.00000000001], 
    'recur_reg_factor' : [0.1], 
    'bias_reg_factor' : [0.1],
    'units_layer2' : [units_layer2],
    'activation_layer2' : [None if units_layer2 is None else 'relu'],
    'dropout_layer2' : [None if units_layer2 is None else 0.1],
    'activation_output' : ['relu'],
    'optim' : ['adam'],
}

company_inputs = ['PETR4.SA']
company_output = 'PETR4.SA'
end_date = '2024-10-30' 
start_date = '2020-01-01'


In [20]:
from sklearn.model_selection import ParameterGrid
from parametrized_training_model import parametrized_training

import mlflow
import mlflow.tensorflow

from sklearn.metrics import root_mean_squared_error, r2_score

mlflow.set_tracking_uri(uri="http://127.0.0.1:7777")
mlflow.set_experiment("LSTM Grid Search Test")

param_grid = ParameterGrid(params)
for param_sample in param_grid:
    (scalerX, scalery, generator_train, generator_test, model) = parametrized_training(company_inputs, company_output, start_date, end_date, train_ratio,
                          horizon_pred, seq_length, batch_size, n_epochs)

    hist = model.fit(generator_train, epochs=n_epochs, validation_data=generator_test)

    X_gen_train, y_gen_train = generator_train[0]
    X_gen_test, y_gen_test = generator_test[0]
    y_pred_scaled = model.predict(X_gen_test)
    y_pred = scalery.inverse_transform(y_pred_scaled.reshape(-1,1))
    y_test = scalery.inverse_transform(y_gen_test.reshape(-1,1))
    # y_pred = y_pred_scaled.reshape(-1,1)
    # y_valid_f = y_gen_test.reshape(-1,1)


    mlflow.tensorflow.autolog()
    with mlflow.start_run():
        mlflow.tensorflow.log_model(model, "model")

        # Calculate RMSE and R² score
        rmse = root_mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        mlflow.log_params(param_sample)

        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("R2-Score", r2)

2024/11/16 19:38:22 INFO mlflow.tracking.fluent: Experiment with name 'LSTM Grid Search Test' does not exist. Creating a new experiment.
[*********************100%***********************]  1 of 1 completed
c:\GitRepos\TC4-Thales\tech_challenge_4\training_model\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.1624 - val_loss: 0.0028
Epoch 2/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0107 - val_loss: 0.0529
Epoch 3/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0077 - val_loss: 0.0182
Epoch 4/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0059 - val_loss: 0.0213
Epoch 5/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0059 - val_loss: 0.0024
Epoch 6/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0022 - val_loss: 0.0051
Epoch 7/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0019 - val_loss: 0.0029
Epoch 8/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0018 - val_loss: 0.0026
Epoch 9/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0036 - val_loss: 0.0042
Epoch 10/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0017 - val_loss: 0.0166
Epoch 11/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0038 - val_loss: 0.0046
Epoch 12/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0

2024/11/16 19:38:42 WARNING mlflow.utils.autologging_utils: MLflow tensorflow autologging is known to be compatible with 2.7.4 <= tensorflow <= 2.17.0, but the installed version is 2.18.0. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a compatible version, or try upgrading MLflow.
2024/11/16 19:38:43 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/11/16 19:38:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/16 19:38:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run gaudy-crab-641 at: http://127.0.0.1:7777/#/experiments/618798653659337455/runs/657a5827f76643f5a3c24c61a60731f7.
2024/11/16 19:38:58 INFO m

Epoch 1/30
30/34 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0683

34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 0.0645 - val_loss: 0.0176
Epoch 2/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0120 - val_loss: 0.1933
Epoch 3/30
33/34 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0258

...

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0012 - val_loss: 0.0016
Epoch 21/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0012 - val_loss: 0.0018
Epoch 22/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0018 - val_loss: 0.0032
Epoch 23/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0019 - val_loss: 0.0038
Epoch 24/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0013 - val_loss: 0.0072
Epoch 25/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0023 - val_loss: 0.0040
Epoch 26/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0015 - val_loss: 0.0019
Epoch 27/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0015 - val_loss: 0.0025
Epoch 28/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0015 - val_loss: 0.0019
Epoch 29/30
31/34 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0011  

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0011 - val_loss: 0.0015
Epoch 30/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0012 - val_loss: 0.0018


2024/11/16 19:40:48 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: unsupported operand type(s) for *: 'int' and 'slice'
2024/11/16 19:40:48 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/11/16 19:40:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/16 19:40:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run victorious-hawk-538 at: http://127.0.0.1:7777/#/experiments/618798653659337455/runs/ffc9de1bd2d742cd948cd13436fc9420.
2024/11/16 19:40:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:7777/#/experiments/618798653659337455.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step


2024/11/16 19:40:58 WARNING mlflow.utils.autologging_utils: MLflow tensorflow autologging is known to be compatible with 2.7.4 <= tensorflow <= 2.17.0, but the installed version is 2.18.0. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a compatible version, or try upgrading MLflow.
2024/11/16 19:40:58 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/11/16 19:41:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/16 19:41:07 INFO mlflow.tracking._tracking_service.client: 🏃 View run nosy-stoat-387 at: http://127.0.0.1:7777/#/experiments/618798653659337455/runs/e34d67c7b0a342158e9fe656c570505b.
2024/11/16 19:41:07 INFO m

In [21]:
# from sklearn.metrics import root_mean_squared_error, r2_score

# rmse = root_mean_squared_error(y_pred, y_valid_f)
# r2 = r2_score(y_pred, y_valid_f)

# print(f'RMSE: {rmse:.4f}')
# print(f'R² Score: {r2:.4f}')

In [22]:
# import matplotlib.pyplot as plt

# # Plot the predicted prices
# plt.plot(range(len(y_gen_train)),y_gen_train, label="Training Data")
# plt.plot(range(len(y_gen_train),len(y_gen_train)+len(y_gen_test)),y_gen_test, label="Test Data")
# plt.xlabel("Days")
# plt.ylabel("Price")
# plt.legend()
# plt.show()

In [23]:
# import matplotlib.pyplot as plt

# # Plot the predicted prices
# plt.plot(y_pred, label="Predicted Prices")
# plt.plot(y_valid_f, label="Real Prices")
# plt.xlabel("Days")
# plt.ylabel("Price")
# plt.legend()
# plt.show()

In [24]:
# import mlflow
# import mlflow.tensorflow

# mlflow.set_tracking_uri(uri="http://127.0.0.1:7777")
# mlflow.set_experiment("LSTM Experiments")

# from sklearn.metrics import root_mean_squared_error, r2_score

# mlflow.tensorflow.autolog()
# with mlflow.start_run():
#     mlflow.tensorflow.log_model(model, "model")

#     # Calculate RMSE and R² score
#     rmse = root_mean_squared_error(y_test, y_pred)
#     r2 = r2_score(y_test, y_pred)

#     print(f'RMSE: {rmse:.4f}')
#     print(f'R² Score: {r2:.4f}')

#     mlflow.log_metric("RMSE", rmse)
#     mlflow.log_metric("R2-Score", r2)

In [25]:
# # Predict using the trained model
# future_steps = 10  # Number of days to predict
# future_data = data[-n_steps:].reshape(-1, n_steps, 1)
# predicted_prices = []

# for i in range(future_steps):
#     prediction = model.predict(future_data)[0, 0]
#     predicted_prices.append(prediction)
#     future_data = np.roll(future_data, -1, axis=1)
#     future_data[0, -1] = prediction

# # Inverse transform the predicted prices to original scale
# predicted_prices = scaler.inverse_transform(np.array(predicted_prices).reshape(-1, 1)).flatten()

In [26]:
# import matplotlib.pyplot as plt

# # Plot the predicted prices
# plt.plot(predicted_prices, label="Predicted Prices")
# plt.xlabel("Days")
# plt.ylabel("Price")
# plt.legend()
# plt.show()